# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
# Q1 & Q2
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

total = 0
counter = 0
for sqrt_value in generator:
    counter += 1
    if counter <= 5:
      total += sqrt_value
    msg = f"Q2 >>> {counter}: {sqrt_value}" if counter == 13 else f"{counter}: {sqrt_value}"
    print(f"{msg}")

print(f"Q1 >>> Total sum for limit = 5: {total}")



1: 1.0
2: 1.4142135623730951
3: 1.7320508075688772
4: 2.0
5: 2.23606797749979
6: 2.449489742783178
7: 2.6457513110645907
8: 2.8284271247461903
9: 3.0
10: 3.1622776601683795
11: 3.3166247903554
12: 3.4641016151377544
Q2 >>> 13: 3.605551275463989
Q1 >>> Total sum for limit = 5: 8.382332347441762


In [6]:
! python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


In [19]:
#Install the dependencies in Jupyter Notebook rather than Google Colab
# %%capture
! pip3 install 'dlt[duckdb]'
! pip3 install pandas
! pip3 install numpy



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [20]:
import dlt
import duckdb

# define the connection to load to.
pipeline = dlt.pipeline(pipeline_name="dlt_workshop", destination='duckdb', dataset_name='people_ds')

In [21]:
#Q3
#1. Load the first generator

data = []

for p in people_1():
  data.append(p)


info = pipeline.run(data,
										table_name="people",
										write_disposition="replace",
                    primary_key="ID")

print(info)

Pipeline dlt_workshop load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_ds
The duckdb destination used duckdb:////Users/crowley/workspace/de-zoomcamp/workshops/dlt/dlt_workshop.duckdb location to store data
Load package 1708096709.415416 is LOADED and contains no failed jobs


In [23]:
# and calculate the sum of ages of all people. Make sure to only load it once.
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT age FROM people'
    ) as table:
        # calling `df` on a cursor, returns the data as a DataFrame
        ages = table.df()
    
ages_sum = ages.sum(0)

print(ages_sum)

age    140
dtype: int64


In [24]:
#2. Append the second generator to the same table as the first.
data = []

for p in people_2():
  data.append(p)


info = pipeline.run(data,
										table_name="people",
										write_disposition="append",
                    primary_key="ID")

print(info)


Pipeline dlt_workshop load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_ds
The duckdb destination used duckdb:////Users/crowley/workspace/de-zoomcamp/workshops/dlt/dlt_workshop.duckdb location to store data
Load package 1708096730.665092 is LOADED and contains no failed jobs


In [25]:
#3. After correctly appending the data, calculate the sum of all ages of people.

with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT age FROM people'
    ) as table:
        # calling `df` on a cursor, returns the data as a DataFrame
        ages = table.df()

        #print(ages)
ages_sum = ages.sum(0)

print(f"Q3 >>> sum of all people ages: {ages_sum}")

Q3 >>> sum of all people ages: age    353
dtype: int64


In [26]:
# Q4

#Re-use the generators from Exercise 2.

#A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.
#Load your first generator first,

data = []

for p in people_1():
  data.append(p)


info = pipeline.run(data,
										table_name="people_q4",
										write_disposition="replace",
                    primary_key="ID")

print(info)


Pipeline dlt_workshop load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_ds
The duckdb destination used duckdb:////Users/crowley/workspace/de-zoomcamp/workshops/dlt/dlt_workshop.duckdb location to store data
Load package 1708096742.560247 is LOADED and contains no failed jobs


In [27]:
# and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

data = []

for p in people_2():
  data.append(p)


info = pipeline.run(data,
										table_name="people_q4",
										write_disposition="merge",
                    primary_key="ID")

print(info)


Pipeline dlt_workshop load step completed in 0.37 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_ds
The duckdb destination used duckdb:////Users/crowley/workspace/de-zoomcamp/workshops/dlt/dlt_workshop.duckdb location to store data
Load package 1708096749.693969 is LOADED and contains no failed jobs


In [28]:
#After loading, you should have a total of 8 records, and ID 3 should have age 33.
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT * FROM people_q4'
    ) as table:
        # calling `df` on a cursor, returns the data as a DataFrame
        people = table.df()

        print(people.sort_values("id"))
people_ages_sum = people["age"].sum(0)

print(f"Q4 >>> sum of all people ages: {people_ages_sum}")

#Question: Calculate the sum of ages of all the people loaded as described above.

   id      name  age    city       _dlt_load_id         _dlt_id occupation
0   1  Person_1   26  City_A  1708096742.560247  ftwaxVJKSXDIZA       None
1   2  Person_2   27  City_A  1708096742.560247  ZZzfzdrpYYuntQ       None
4   3  Person_3   33  City_B  1708096749.693969  07+vDAqMvog+aw      Job_3
7   4  Person_4   34  City_B  1708096749.693969  gRqed1HM4oJ5vg      Job_4
2   5  Person_5   35  City_B  1708096749.693969  m/2R0fwFsY0NaA      Job_5
5   6  Person_6   36  City_B  1708096749.693969  O59CaIpZGhJp+g      Job_6
6   7  Person_7   37  City_B  1708096749.693969  pjiXyMZvPd6ylA      Job_7
3   8  Person_8   38  City_B  1708096749.693969  goQNnxtmTqX/vw      Job_8
Q4 >>> sum of all people ages: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX